In [ ]:
!apt install swig
!pip install orthnet

In [1]:

import torch
import numpy as np
import torch.optim as optim
from torch.autograd import Variable
from torch import randn
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn

import random, os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd

from orthnet import Legendre, Chebyshev

%matplotlib inline

GPU_MODE = True

In [2]:
if GPU_MODE:
  cuda0 = torch.device('cuda:0')
  print(torch.cuda.device_count(), torch.cuda.get_device_name(0))
  torch.cuda.init()

1 NVIDIA GeForce GTX 1070


In [3]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [4]:
class LegendreActivation(nn.Module):
    def __init__(self,degree):
        super().__init__()
        self.degree = degree
        self.data = None
        
        self.D = torch.zeros((degree , degree )) 
        for i in range(degree):
          for j in range(0 , i):
            if (i + j) % 2 == 1:
              self.D[i, j] = 2 * (j + 1) - 1
    
    def forward(self, X):              
      data = Legendre(X, self.degree).tensor
      self.data = data
      return data

    def backward(self,):
      return (self.D @ (self.data).T).T


class LegendreBlock(nn.Module):
    def __init__(self, n_input, degree):        
        super().__init__()
        self.degree = degree - 1
        self.n_input = n_input
        self.linear = nn.Linear(self.n_input, 1).double()
        self.tanh = nn.Tanh().double()
        self.Legendre = LegendreActivation(self.degree)

    def forward(self, X):      
      X = self.tanh(self.linear(X))      
      data = self.Legendre(X)
      return data

class ChebyshevActivation(nn.Module):
    def __init__(self,degree):
        super().__init__()
        self.degree = degree
        self.data = None
        
        self.D = torch.zeros((degree , degree )) 
        for i in range(degree):
          for j in range(0 ,i):
            if (i+j) % 2 == 1:
              self.D[i, j] = 2 * i
              if j == 0:
                self.D[i, j] = self.D[i, j]/2.0
    def forward(self, X):              
      data = Chebyshev(X, self.degree).tensor
      self.data = data
      return data

    def backward(self,):
      return (self.D @ (self.data).T).T

class ChebyshevBlock(nn.Module):
    def __init__(self, n_input, degree):        
        super().__init__()
        self.degree = degree - 1
        
        self.n_input = n_input
        self.linear = nn.Linear(self.n_input, 1).double()
        self.tanh = nn.Tanh().double()
        self.Chebyshev = ChebyshevActivation(self.degree)


    def forward(self, X):      
      X = self.tanh(self.linear(X))      
      data = self.Chebyshev(X)
      
      return data

In [5]:
def dy_dx(y, x):
  return torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y), create_graph=True)[0]

def d2y_dx2(y, x):
  return dy_dx(dy_dx(y,x), x)

def d3y_dx3(y, x):
  return dy_dx(d2y_dx2(y,x), x)

In [6]:
domain = [0, 6]
n_discretization = 3000 * domain[1] - domain[0]

In [7]:
n_input = 1
n_output = 1
eps = 1e-10

In [8]:
if GPU_MODE:
  x = torch.linspace(domain[0] + eps, domain[1] - eps, n_discretization, dtype=torch.double, device=cuda0).reshape(-1,1)
  x = Variable(x, requires_grad=True).double()
else:
  x = torch.linspace(domain[0] + eps, domain[1] - eps, n_discretization, dtype=torch.double).reshape(-1,1)
  x = Variable(x, requires_grad=True).double()

In [9]:
mlp = nn.Sequential(  
  LegendreBlock(n_input, 16),
  nn.Linear(16, n_output)
).double()


pytorch_total_params = sum(p.numel() for p in mlp.parameters() if p.requires_grad)
pytorch_total_params

19

In [10]:
if GPU_MODE:
  mlp.to('cuda:0')

In [11]:
def get_loss(x, ret_res=False):
  y = mlp(x)
  y_x = dy_dx(y, x)
  y_xx = dy_dx(y_x, x)
  y_xxx = dy_dx(y_xx, x)


  """    
  Generall Falkner-Skan Eq.: 
    f''' + α ff'' + β(1 - (f')^2) = 0 ,
    f(0) = f'(0) = 0, f'(∞) = 1
  
  Homann Flow = (α, β) = (2, 1)  
  """
  alpha = 2
  beta = 1

  residual = (y_xxx) + (alpha * y_xx * y) + (beta * (1 - y_x**2))


  # boundaries same for all equations
  boundary1 = y[0]
  boundary2 = y_x[0]
  boundary3 = y_x[-1] - 1

  loss = (residual**2).mean() + boundary1**2 + boundary2**2 + boundary3**2
  return (loss, residual) if ret_res else loss

In [12]:
def closure():
  loss = get_loss(x)
  optimizer.zero_grad()
  loss.backward()
  return loss

In [13]:
#TODO
optimizer = optim.Adam(list(mlp.parameters()), lr=0.05, betas=(0.9, 0.999), eps=1e-32)
previous = 0
losses = []
epoch_Adam = 100
epoch_LBFGS = 10000
for i in range(epoch_Adam):
  loss = get_loss(x)
  
  if i % 2 == 0:        
    print('Step: %03d loss = %016.10f' % (i, loss))        
  
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  losses.append(loss.detach().cpu().numpy())


optimizer = optim.LBFGS(list(mlp.parameters()), lr = 0.01)
previous = 0
for i in range(epoch_LBFGS):
  loss = get_loss(x)
  if i % 2 == 0:        
    print('Step: %03d loss = %016.10f' % (i+epoch_Adam, loss))
    if abs(previous - loss) < 1e-10:
        print('converged')
        break
    
    previous = loss
  
  losses.append(loss.detach().cpu().numpy())
  optimizer.step(closure)

print("Final loss = %.2e" % get_loss(x))

Step: 000 loss = 00012.4102512127
Step: 002 loss = 00005.9356055314
Step: 004 loss = 00003.7653164648
Step: 006 loss = 00001.9403460795
Step: 008 loss = 00001.5113232739
Step: 010 loss = 00001.5213850682
Step: 012 loss = 00001.5115711970
Step: 014 loss = 00001.4493725724
Step: 016 loss = 00001.3716196789
Step: 018 loss = 00001.2810280602
Step: 020 loss = 00001.1724793691
Step: 022 loss = 00001.0603390437
Step: 024 loss = 00000.9606681846
Step: 026 loss = 00000.8697934717
Step: 028 loss = 00000.7723694808
Step: 030 loss = 00000.6643343583
Step: 032 loss = 00000.5621796822
Step: 034 loss = 00000.4920084819
Step: 036 loss = 00000.4626164768
Step: 038 loss = 00000.4472949942
Step: 040 loss = 00000.4188602520
Step: 042 loss = 00000.3891087077
Step: 044 loss = 00000.3753184997
Step: 046 loss = 00000.3697521801
Step: 048 loss = 00000.3583094229
Step: 050 loss = 00000.3391847518
Step: 052 loss = 00000.3212812596
Step: 054 loss = 00000.3088355057
Step: 056 loss = 00000.2948783871
Step: 058 loss

In [15]:
# calculate f''(0)
# # make zero Tensor from x Tensor
zerox = x.clone()
zerox[0][0] = 0
f_xx_0 = d2y_dx2(mlp(zerox), zerox)[0]
print("f''(0) = {}".format(f_xx_0[0]))


#  Standard = 1.3119377173

f''(0) = 1.3123043247236872


In [ ]:
mlp.to('cpu')
plt.figure(figsize=(10, 5))
domain = x.cpu().detach().numpy().flatten()

res= get_loss(x.cpu(), ret_res=True)[1].detach().numpy()
plt.xlabel('x')
plt.ylabel('Residual')
plt.plot(domain, res)
plt.savefig('residual-loss.eps', bbox_inches='tight', format='eps')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(np.log(losses))
plt.xlabel('epoch')
plt.ylabel('log(loss)')

plt.savefig('loss.eps', bbox_inches='tight', format='eps')